In [ ]:
import os
from dotenv import load_dotenv
from pyprojroot import here
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('GITHUB_TOKEN')
import httpx

True

In [ ]:
os.environ['OPENAI_API_KEY'] = os.getenv('GITHUB_TOKEN')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')

**1.1 RAG tool design**

In [ ]:
from langchain_qdrant import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_core.tools import tool
from qdrant_client import QdrantClient

EMBEDDING_MODEL = 'text-embedding-3-small'
VECTORDB_DIR = 'data/clinical_notes_vectordb'
QDRANT_URL = None  # Set to a URL for remote Qdrant, or None for local
K = 2

@tool
def lookup_clinical_notes(query: str)->str:
    """Search among clinical notes to check relevant information."""
    client = QdrantClient(url=QDRANT_URL, path=str(here(VECTORDB_DIR)) if not QDRANT_URL else None)
    vectordb = Qdrant(
        client=client,
        collection_name='clinical-notes-qdrant',
        embeddings=OpenAIEmbeddings(
            model=EMBEDDING_MODEL,
            base_url='https://models.github.ai/inference',
            api_key=os.getenv('GITHUB_TOKEN')
        )
    )
    docs = vectordb.similarity_search(query=query, k=K)
    return '\n\n'.join([doc.page_content for doc in docs])

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langchain_openai import ChatOpenAI
from agent_graph.tool_travel_sqlagent import query_travel_sqldb
from agent_graph.tool_tavily_search import load_tavily_search_tool
from agent_graph.load_tools_config import LoadToolsConfig
from agent_graph.agent_backend import State, BasicToolNode, route_tools, plot_agent_schema

TOOLS_CFG = LoadToolsConfig()

primary_llm = ChatOpenAI(
    model=TOOLS_CFG.primary_agent_llm,
    temperature=TOOLS_CFG.primary_agent_llm_temperature,
    base_url='https://models.github.ai/inference',
    api_key=os.getenv('GITHUB_TOKEN')
)
graph_builder = StateGraph(State)
# Load tools with their proper configs
search_tool = load_tavily_search_tool(TOOLS_CFG.tavily_search_max_results)
tools = [
    search_tool,
    lookup_clinical_notes,
    query_travel_sqldb,
]
# Tell the LLM which tools it can call
primary_llm_with_tools = ChatOpenAI(
    base_url='https://models.github.ai/inference',
    api_key=os.getenv('GITHUB_TOKEN'),
    model='gpt-4o-mini'
)

def chatbot(state: State):
    """Executes the primary language model with tools bound and returns the generated message."""
    return {'messages': [primary_llm_with_tools.invoke(state['messages'])]}

graph_builder.add_node('chatbot', chatbot)
tool_node = BasicToolNode(
    tools=[
        search_tool,
        lookup_clinical_notes,
        query_travel_sqldb,
    ])
graph_builder.add_node('tools', tool_node)
graph_builder.add_conditional_edges(
    'chatbot',
    route_tools,
    {'tools': 'tools', '__end__': '__end__'},
)
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
plot_agent_schema(graph)

**Test the graph**

In [ ]:
config = {'configurable': {'thread_id': TOOLS_CFG.thread_id}}
events = graph.stream(
    {'messages': [('user', 'What are the symptoms mentioned in the clinical notes?')]}, config, stream_mode='values'
)
for event in events:
    event['messages'][-1].pretty_print()